# R_16 Create Data for final Data Analysis

In [1]:
import pandas as pd
import json

In [2]:
sudo_election_data = pd.read_csv('../data/curated/DataAnalysis/By_Electorate_SUDO_Political_Data.csv')

In [3]:
sudo_election_data

,DivisionAb,DivisionName,DivisionId,state,WinningParty2022,WinningParty2019,WinningPercent2022,WinningPercent2019,ChangedWinningParty,ChangeInWinningPercent,2022WinningPartyOwnChangeInPercent,avg_age,avg_educ_years,avg_weekly_income,erp,net_migration
0,BANK,Banks,103.0,NSW,LP,LP,0.562619,0.514405,0,0.048214,0.048214,47.683199,11.178675,1854.413529,265134.252037,-3262.304824
1,BART,Barton,104.0,NSW,ALP,ALP,0.594147,0.583008,0,0.011139,0.011139,47.258842,11.221474,1925.585201,225239.181790,-2010.603275
2,BENN,Bennelong,105.0,NSW,LP,LP,0.569119,0.597164,0,-0.028045,-0.028045,46.541816,11.475803,2083.386432,167623.685734,-1931.982319
3,BERO,Berowra,106.0,NSW,LP,LP,0.656487,0.664454,0,-0.007967,-0.007967,49.318139,11.454072,2381.047743,165880.753324,292.738308
4,BLAX,Blaxland,107.0,NSW,ALP,ALP,0.647201,0.694836,0,-0.047636,-0.047636,46.840966,11.059208,1718.150609,338945.555525,-5110.648408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,OCON,O'Connor,243.0,WA,LP,LP,0.654147,0.650413,0,0.003734,0.003734,44.697091,10.660585,1395.180665,4508.688877,-60.496850
147,PEAR,Pearce,244.0,WA,LP,LP,0.551914,0.536263,0,0.015652,0.015652,45.576736,11.157276,1798.550239,207044.908074,1154.886979
148,PERT,Perth,245.0,WA,ALP,ALP,0.532269,0.533319,0,-0.001050,-0.001050,46.654956,11.329518,1881.142261,95350.185618,-844.044273
149,SWAN,Swan,247.0,WA,LP,LP,0.532241,0.535920,0,-0.003679,-0.003679,47.115725,11.248214,1818.224303,59821.560093,-378.393021


# Create API

In [4]:
import requests
import time

username = 'group9_admin'
password = 'group9_H1'

In [5]:
def make_get_request(username, password, url, params = {}):
    """ Helper to make the GET request using the 
            username - str
            password - str
            url - str (e.g. http://172.26.133.251:5984/tweet_database/_design/date/_view/date_electorate__nsentiment)
            params - dict (e.g. {'group': 'True'})"""
    
    start = time.time()

    # Set the authentication credentials
    auth = (username, password)

    # Send GET request using requests library
    response = requests.get(url, params=params, auth=auth)
    
    end = time.time()
    print(end-start)

    if response.ok:
        return response.json()
    else:
        return 'Error: '+ response.status_code

In [6]:
def GET_date_electorate__avgsent__pol(startdate, enddate, username, password):
    url = 'http://172.26.133.251:5984/tweet_database/_design/date/_view/date_electorate__avgsent__pol'

    # Set the parameters for the GET request
    params = {
        'group': 'True',
        'startkey': f'["{startdate}"]',
        'endkey': f'["{enddate}"]'
    }

    returned_data_json = make_get_request(username, password, url, params)
    
    return returned_data_json

In [7]:
month3 = GET_date_electorate__avgsent__pol('2022-02-21', '2022-05-22', username, password) # Pre Election
month2 = GET_date_electorate__avgsent__pol('2022-03-21', '2022-05-22', username, password) # Pre Election
month1 = GET_date_electorate__avgsent__pol('2022-04-21', '2022-05-22', username, password) # Pre Election
week3 = GET_date_electorate__avgsent__pol('2022-05-01', '2022-05-22', username, password) # Pre Election
week2 = GET_date_electorate__avgsent__pol('2022-05-07', '2022-05-22', username, password) # Pre Election
week1 = GET_date_electorate__avgsent__pol('2022-05-14', '2022-05-22', username, password) # Pre Election

4.609930992126465
3.5614428520202637
2.4681878089904785
1.6969871520996094
1.1360621452331543
0.9116730690002441


In [8]:
def clean_data(date_electorate__avgsent__pol):
    electorate_avg_sent_pol_sum_count = {}

    for i in range(len(date_electorate__avgsent__pol['rows'])):
        
        electorate = date_electorate__avgsent__pol['rows'][i]['key'][1]
        if electorate not in electorate_avg_sent_pol_sum_count:
            electorate_avg_sent_pol_sum_count[electorate] = [0, 0]
        
        electorate_avg_sent_pol_sum_count[electorate][0] += date_electorate__avgsent__pol['rows'][i]['value']['sum']
        electorate_avg_sent_pol_sum_count[electorate][1] += date_electorate__avgsent__pol['rows'][i]['value']['count']

    electorate_avg_sent = {key:electorate_avg_sent_pol_sum_count[key][0]/electorate_avg_sent_pol_sum_count[key][1] for key in electorate_avg_sent_pol_sum_count}
    return electorate_avg_sent


In [9]:
month3_cleaned = clean_data(month3)
month2_cleaned = clean_data(month2)
month1_cleaned = clean_data(month1)
week3_cleaned = clean_data(week3)
week2_cleaned = clean_data(week2)
week1_cleaned = clean_data(week1)

In [10]:
sentiment_df1 = pd.DataFrame({'electorate': list(month3_cleaned.keys()), 'month3':list(month3_cleaned.values())})
sentiment_df2 = pd.DataFrame({'electorate': list(month2_cleaned.keys()), 'month2':list(month2_cleaned.values())})
sentiment_df3 = pd.DataFrame({'electorate': list(month1_cleaned.keys()), 'month1':list(month1_cleaned.values())})
sentiment_df4 = pd.DataFrame({'electorate': list(week3_cleaned.keys()), 'week3':list(week3_cleaned.values())})
sentiment_df5 = pd.DataFrame({'electorate': list(week2_cleaned.keys()), 'week2':list(week2_cleaned.values())})
sentiment_df6 = pd.DataFrame({'electorate': list(week1_cleaned.keys()), 'week1':list(week1_cleaned.values())})

In [11]:
sentiment_df6

,electorate,week1
0,Adelaide,0.135519
1,Aston,0.078973
2,Ballarat,0.263178
3,Barker,0.099432
4,Bass,-0.040631
...,...,...
131,Moore,0.718400
132,Oxley,0.709400
133,Robertson,0.430300
134,Solomon,0.796400


In [12]:
sudo_election_data_with_sentiment = sudo_election_data.merge(right = sentiment_df1, how = 'left', left_on='DivisionName', right_on='electorate')
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.drop('electorate', axis = 1)
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.merge(right = sentiment_df2, how = 'left', left_on='DivisionName', right_on='electorate')
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.drop('electorate', axis = 1)
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.merge(right = sentiment_df3, how = 'left', left_on='DivisionName', right_on='electorate')
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.drop('electorate', axis = 1)
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.merge(right = sentiment_df4, how = 'left', left_on='DivisionName', right_on='electorate')
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.drop('electorate', axis = 1)
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.merge(right = sentiment_df5, how = 'left', left_on='DivisionName', right_on='electorate')
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.drop('electorate', axis = 1)
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.merge(right = sentiment_df6, how = 'left', left_on='DivisionName', right_on='electorate')
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.drop('electorate', axis = 1)

In [13]:
sudo_election_data_with_sentiment = sudo_election_data_with_sentiment.fillna(0)

In [14]:
sudo_election_data_with_sentiment

,DivisionAb,DivisionName,DivisionId,state,WinningParty2022,WinningParty2019,WinningPercent2022,WinningPercent2019,ChangedWinningParty,ChangeInWinningPercent,...,avg_educ_years,avg_weekly_income,erp,net_migration,month3,month2,month1,week3,week2,week1
0,BANK,Banks,103.0,NSW,LP,LP,0.562619,0.514405,0,0.048214,...,11.178675,1854.413529,265134.252037,-3262.304824,0.493040,0.293900,0.293900,0.293900,0.293900,0.293900
1,BART,Barton,104.0,NSW,ALP,ALP,0.594147,0.583008,0,0.011139,...,11.221474,1925.585201,225239.181790,-2010.603275,-0.075429,0.370700,0.299700,0.000000,0.000000,0.000000
2,BENN,Bennelong,105.0,NSW,LP,LP,0.569119,0.597164,0,-0.028045,...,11.475803,2083.386432,167623.685734,-1931.982319,0.217421,0.156362,0.224779,0.316291,0.233967,0.138000
3,BERO,Berowra,106.0,NSW,LP,LP,0.656487,0.664454,0,-0.007967,...,11.454072,2381.047743,165880.753324,292.738308,0.359983,0.265700,0.469355,0.505611,0.555829,0.555829
4,BLAX,Blaxland,107.0,NSW,ALP,ALP,0.647201,0.694836,0,-0.047636,...,11.059208,1718.150609,338945.555525,-5110.648408,-0.035998,0.070033,0.035215,0.028444,0.124330,0.096617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,OCON,O'Connor,243.0,WA,LP,LP,0.654147,0.650413,0,0.003734,...,10.660585,1395.180665,4508.688877,-60.496850,0.039615,0.043300,0.071502,0.078735,0.081709,0.075049
147,PEAR,Pearce,244.0,WA,LP,LP,0.551914,0.536263,0,0.015652,...,11.157276,1798.550239,207044.908074,1154.886979,0.189727,0.189727,0.043664,0.025880,0.403533,0.210750
148,PERT,Perth,245.0,WA,ALP,ALP,0.532269,0.533319,0,-0.001050,...,11.329518,1881.142261,95350.185618,-844.044273,0.421430,0.366368,0.434719,0.491925,0.462421,0.363414
149,SWAN,Swan,247.0,WA,LP,LP,0.532241,0.535920,0,-0.003679,...,11.248214,1818.224303,59821.560093,-378.393021,0.399142,0.426970,0.500318,0.487871,0.481411,0.190060


In [15]:
sudo_election_data_with_sentiment.to_csv('../data/curated/DataAnalysis/By_Electorate_SUDO_Political_Sentiment_Data.csv', index = False)

In [16]:
# electorates = list(electorate_avg_sent.keys())
# electorates.sort()

# with open('../data/curated/GeoJson/electorate_list.json', 'w') as f:
#     json.dump(electorates, f)